In [1]:
import geopandas as gpd
import pandas as pd
from utils import gdf_operations, functions

2025-01-23 14:31:58.148 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
distrito =gdf_operations.get_dados('distrito')
subbac = gdf_operations.get_dados('subbac')
subpref = gdf_operations.get_dados('subpref')
fcu = gdf_operations.get_dados('fcu')

In [3]:
cd_subbac = gdf_operations.find_subbac_name(subbac, 'cd_')
nm_subbac = gdf_operations.find_subbac_name(subbac, 'nm_')

unidades_list = [
    ('subbac', "Sub Bacias Hidrográficas", cd_subbac, nm_subbac),
    ('subpref', "Subprefeituras", "cd_subpref" , 'nm_subpref'),
    ('distrito', "Distritos", 'cd_distrit', 'nm_distrit'),
    ('fcu', "Favelas e Comunidades Urbanas", 'cd_fcu', 'nm_fcu')
    
]
unidades_df = pd.DataFrame(unidades_list, columns=['gdf_name', 'name', 'column_cd', 'column_name'])


Vou começar pelo fcu para ver se dá certo.

# FCU

In [4]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:108: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [5]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

In [6]:
overlay_gdf = gdf_operations.overlay_intersec('distrito', gdf_unidade, 'cd_fcu')

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:97: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


# Distritos

Agora vamos tentar com os distritos

In [7]:
distrito_sorted = gdf_operations.create_gdf_sorted(
    distrito, 'distrito'
)

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:108: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [8]:
choice_unidade = 'distrito'
gdf_unidade = distrito_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

In [9]:
overlay_gdf = gdf_operations.overlay_intersec(
    'subpref', gdf_unidade, 'cd_distrit')

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:97: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


# Loop por intersec

## Vamos Começar com o FCU mesmo

In [10]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:108: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [11]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

In [12]:
print(unidades_df[unidades_df['gdf_name']=='fcu'])

  gdf_name                           name column_cd column_name
3      fcu  Favelas e Comunidades Urbanas    cd_fcu      nm_fcu


In [13]:
index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == choice_unidade
        ].index[0]
    )

name_gdf_unidade = choice_unidade
batatinha = pd.DataFrame()

cd_unidade = gdf_operations.find_gdf_name(choice_undia)

overlay_gdf = gpd.GeoDataFrame()

for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )
            print(overlay_gdf.columns)
            
            gdf_unidade[cd_unidade] = gdf_unidade[cd_unidade].astype(str)
            overlay_gdf[cd_intersec] = overlay_gdf[cd_intersec].astype(int).astype(str)
            overlay_gdf[cd_unidade] = overlay_gdf[cd_unidade].astype(str)


            gdf_unidade = pd.merge(gdf_unidade, overlay_gdf[[cd_unidade, cd_intersec]], on=cd_unidade, how='left')
           
            




C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:97: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'nm_bacia_h', 'geometry', 'area'],
      dtype='object')


C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:97: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'cd_subpref', 'nm_subpref', 'geometry', 'area'],
      dtype='object')


C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:97: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'cd_subpref', 'cd_distrit', 'nm_distrit', 'geometry', 'area'],
      dtype='object')


In [14]:
gdf_unidade

,cd_fcu,nm_fcu,pop_fcu,dom_fcu,con_fcu,geometry,cd_identif,cd_subpref,cd_distrit
0,35503080069,Jardim Miliunas,1159,411,162260,"POLYGON ((359999.294 7400461.018, 359969.451 7...",718,24,36
1,35503082222,Avenida Arraias do Araguaia,69,27,9660,"POLYGON ((346540.261 7391411.884, 346562.181 7...",645,26,4
2,35503081646,Pavanas,161,58,22540,"POLYGON ((331961.694 7381437.08, 331927.976 73...",582,16,22
3,35503081067,Jardim São Carlos / Gleba 2,1335,496,186900,"POLYGON ((356033.213 7393960.866, 356024.025 7...",706,28,31
4,35503081121,Jardim Jaraguá II,260,118,36400,"POLYGON ((317018.47 7405175.928, 316992.312 74...",740,1,3
...,...,...,...,...,...,...,...,...,...
1354,35503080507,São Francisco,1185,413,165900,"POLYGON ((332139.677 7377746.652, 332133.391 7...",560,16,58
1355,35503080372,Jardim das Rosas,1181,445,165340,"POLYGON ((317588.774 7382183.16, 317570.598 73...",620,17,19
1356,35503080334,Souza Dantas,1749,742,244860,"POLYGON ((330614.329 7383829.698, 330595.928 7...",585,15,38
1357,35503081241,Sete de Setembro II,1276,461,178640,"POLYGON ((328106.645 7369890.12, 328127.518 73...",547,19,30


In [15]:
subbac.columns

Index(['cd_identif', 'sg_fonte_o', 'qt_area', 'nm_bacia_h', 'cd_identi0',
       'nm_bacia_0', 'pop_total', 'dom_total', 'densidade', 'consumo_di',
       'geometry'],
      dtype='object')

In [16]:
overlay_gdf = gdf_operations.overlay_intersec( 
    'distrito', 
    gdf_unidade, 
    'fcu'
)

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:97: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


In [17]:
overlay_gdf

,cd_fcu,nm_fcu,pop_fcu,dom_fcu,con_fcu,cd_identif,cd_subpref,cd_distrit_1,cd_distrit_2,nm_distrit,geometry,area
0,35503080492,Paraisópolis,58361,22801,8170540,620,17,83,83,VILA ANDRADE,"POLYGON ((323596.076 7386707.895, 323581.11 73...",820817.510092
1,35503082071,Parque das Flores / Jardim Continental / COHAB,12309,4687,1723260,645,30,75,75,SAO RAFAEL,"POLYGON ((352822.215 7384454.952, 352808.483 7...",783919.095422
2,35503081076,Jardim Nova Harmonia,20508,7693,2871120,645,30,75,75,SAO RAFAEL,"POLYGON ((352339.197 7387198.413, 352339.361 7...",780700.45498
3,35503080752,Safira,15285,5718,2139900,713,23,44,44,JARDIM HELENA,"POLYGON ((356300.815 7401815.508, 356302.792 7...",740817.467349
4,35503081836,Cidade Ipava,8625,3370,1207500,580,18,43,43,JARDIM ANGELA,"POLYGON ((320574.485 7374405.566, 320600.172 7...",669664.03325
...,...,...,...,...,...,...,...,...,...,...,...,...
1354,35503081644,Ascendino Reis,125,45,17500,602,12,77,77,SAUDE,"POLYGON ((331613.151 7388940.082, 331613.129 7...",2238.50924
1355,35503081184,Banduíra,123,51,17220,678,7,89,89,VILA MARIA,"POLYGON ((340493.606 7398373.512, 340493.289 7...",1989.281926
1356,35503081686,Alegre,191,60,26740,645,26,20,20,CARRAO,"POLYGON ((341414.993 7396204.278, 341392.027 7...",1850.103906
1357,35503081506,Intercontinental,52,25,7280,620,10,94,94,VILA SONIA,"POLYGON ((321295.14 7388849.188, 321298.614 73...",1513.528903


In [18]:

def intersec_unidades(
    unidades_df,
    choice_unidade,
    gdf_unidade,
    name_column_unidade
)-> gpd.GeoDataFrame:

    gdf_unidade = create_gdf_sorted(gdf=gdf_unidade, name_gdf=choice_unidade)
    index_unidade = (
        unidades_df[
            unidades_df['name'] == choice_unidade
        ].index[0]
    )
    resultados = []
    gdf_final_unidade = gpd.GeoDataFrame()
    
    for i, row in unidades_df.iterrows():
        if i< index_unidade:
            polenta = overlay_intersec(unidades_df, choice_unidade, gdf_unidade, name_column_unidade)
            return polenta

